In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from imdb import Cinemagoer

In [3]:
import os
import sys
import inspect

currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0, parentdir) 

In [4]:
from profanity_detector.get_data import movie_data, get_all_movie_data, run_and_display, get_all_movie_data

In [5]:
from profanity_detector.feature_data_enrichment import get_movie_location, get_set_location, get_coordinates, enrich_locations
from geopy.geocoders import Nominatim

In [6]:
import nltk

In [7]:
import spacy

In [8]:
nltk.downloader.download('maxent_ne_chunker')
nltk.downloader.download('words')
nltk.downloader.download('treebank')
nltk.downloader.download('maxent_treebank_pos_tagger')
nltk.downloader.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /home/tomershtein/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     /home/tomershtein/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package treebank to
[nltk_data]     /home/tomershtein/nltk_data...
[nltk_data]   Package treebank is already up-to-date!
[nltk_data] Downloading package maxent_treebank_pos_tagger to
[nltk_data]     /home/tomershtein/nltk_data...
[nltk_data]   Package maxent_treebank_pos_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     /home/tomershtein/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/tomershtein/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [9]:
! python -m spacy download en_core_web_sm

     |████████████████████████████████| 13.9 MB 16.7 MB/s            
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


# Location tagger

In [10]:
import locationtagger

In [11]:
movie_meta, quotes_df, reviews_df, locations_df = movie_data('machete')

In [12]:
location = locations_df['locations'][2]

In [13]:
location

'Troublemaker Studios - 4900 Old Manor Road, Austin, Texas, USA::(studio)'

# preparing the data

In [14]:
def get_movie_location(location):
    if '::' in location:
        splitlist = location.split('::')
        movie_location = splitlist[1]
            
    else:
        movie_location = ''
           
    return movie_location

In [15]:
def get_set_location(location):
    if '::' in location:
        splitlist = location.split('::')
        set_location = splitlist[0]
    
    else:
        set_location = location
    
    
    return set_location 

In [16]:
locations_df['movie_location'] = locations_df.apply(lambda x: get_movie_location(x['locations']),
                                                                     axis=1 )

In [17]:
locations_df['set_location'] = locations_df.apply(lambda x: get_set_location(x['locations']),
                                                                     axis=1 )

In [71]:
locations_df

,locations,imdb_id,title,movie_location,set_location,latitude,longitude
0,"St. Mary's Church - 203 E. 10th St. Austin, Te...",0985694,Machete,,"St. Mary's Church - 203 E. 10th St. Austin, Te...",NaN,NaN
1,"Austin, Texas, USA",0985694,Machete,,"Austin, Texas, USA",NaN,NaN
2,"Troublemaker Studios - 4900 Old Manor Road, Au...",0985694,Machete,(studio),"Troublemaker Studios - 4900 Old Manor Road, Au...",NaN,NaN
3,"Texas, USA",0985694,Machete,,"Texas, USA",NaN,NaN
4,USA,0985694,Machete,,USA,NaN,NaN


In [19]:
locations_df['set_location'][2]

'Troublemaker Studios - 4900 Old Manor Road, Austin, Texas, USA'

# geograpy

In [20]:
import geograpy

In [21]:
from geopy.geocoders import Nominatim

In [22]:
! geograpy-nltk

[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /home/tomershtein/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     /home/tomershtein/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package treebank to
[nltk_data]     /home/tomershtein/nltk_data...
[nltk_data]   Package treebank is already up-to-date!
[nltk_data] Downloading package maxent_treebank_pos_tagger to
[nltk_data]     /home/tomershtein/nltk_data...
[nltk_data]   Package maxent_treebank_pos_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     /home/tomershtein/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/tomershtein/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


value_when_true if condition, else value_when_false

In [23]:
loc_list = list(locations_df['set_location'])

In [24]:
loc_list

["St. Mary's Church - 203 E. 10th St. Austin, Texas, USA",
 'Austin, Texas, USA',
 'Troublemaker Studios - 4900 Old Manor Road, Austin, Texas, USA',
 'Texas, USA',
 'USA']

In [75]:
locations_df = locations_df.drop(['latitude','longitude'], axis=1)

In [76]:
locations_df

,locations,imdb_id,title
0,"St. Mary's Church - 203 E. 10th St. Austin, Te...",0985694,Machete
1,"Austin, Texas, USA",0985694,Machete
2,"Troublemaker Studios - 4900 Old Manor Road, Au...",0985694,Machete
3,"Texas, USA",0985694,Machete
4,USA,0985694,Machete


In [26]:
for location in locations_df['set_location']:
    print(location, get_coordinates(location))    

St. Mary's Church - 203 E. 10th St. Austin, Texas, USA (38.1903261, -76.42523646075932)
Austin, Texas, USA (30.2711286, -97.7436995)
Troublemaker Studios - 4900 Old Manor Road, Austin, Texas, USA (30.2711286, -97.7436995)
Texas, USA (31.2638905, -98.5456116)
USA (39.7837304, -100.445882)


In [27]:
for location in locations_df['set_location']:
    print(location, mega_coordinates(location)) 

NameError: name 'mega_coordinates' is not defined

In [30]:
location_df2['movie_location'] = location_df2.apply(lambda x: get_movie_location(x['locations']), axis=1)

In [31]:
location_df2['set_location'] = location_df2.apply(lambda x: get_set_location(x['locations']), axis=1)

In [69]:
location_df2[['latitude','longitude']] = location_df2.apply(lambda x: get_coordinates(x['set_location']), axis=1, result_type='expand')

In [77]:
enrich_locations(locations_df)

,locations,imdb_id,title,movie_location,set_location,latitude,longitude
0,"St. Mary's Church - 203 E. 10th St. Austin, Te...",0985694,Machete,,"St. Mary's Church - 203 E. 10th St. Austin, Te...",38.190326,-76.425236
1,"Austin, Texas, USA",0985694,Machete,,"Austin, Texas, USA",30.271129,-97.743700
2,"Troublemaker Studios - 4900 Old Manor Road, Au...",0985694,Machete,(studio),"Troublemaker Studios - 4900 Old Manor Road, Au...",30.271129,-97.743700
3,"Texas, USA",0985694,Machete,,"Texas, USA",31.263890,-98.545612
4,USA,0985694,Machete,,USA,39.783730,-100.445882
